# Расчет метрики DCG@K

Входные данные:
- некий запрос **Q** (сейчас не важно какой именно)
- коллекция из N текстовых документов **D_i**, i = 1, ..., N (N может быть большим)
- для каждого документа известна его оценка релевантности **rel_i** относительно запроса **q**

Будем обозначать документы строками вида "d1", "d2", ... (на практике обычно употребляются численные docID, но мы тут упростим себе жизнь).

Предположим, что релевантность оценивается по 3-х балльной шкале:
- 1 - не-релевантный документ
- 2 - полезный документ
- 3 - полностью релевантный документ

Импортируем модули которые нам понадобятся впоследствии:

In [67]:
import math
from catboost import utils

Предположим, что поисковая система в ответ на запрос **Q** выдала 5 документов в следующем порядке:<br>
1. d4
2. d2
3. d1
4. d5
5. d3

Будем хранить поисковую выдачу в списке **search_results**:

In [68]:
search_results = ["d4", "d2", "d1", "d5", "d3"]

Предположим, что нам известны оценки релевантности этих документов.<br>
Будем хранить их словаре **doc2rel**:

In [69]:
doc2rel = {"d1": 2, "d2": 2, "d3": 2, "d4": 3, "d5": 1}

## Считаем DCG@K самостоятельно

Вспомним формулу DCG:<br>
<br>
DCG@K = SUM(2^rel_i - 1 / log(1+i)), где<br>
<br>
i - позиция, на которой находится документ в поисковой выдаче (номера позиций начинаются с 1 и заканчиваются K)<br>
rel_i - оценка релевантности документа на i-й позиции


Чтобы рассчитать DCG, нам будет удобно преобразовать список позиция -> документ в список позиция -> оценка:

In [70]:
def convert_results(search_results, doc2rel):
    rels = []
    for doc in search_results:
        rel_i = doc2rel[doc]
        rels.append(rel_i)
    return rels

In [71]:
y = convert_results(search_results, doc2rel)
print(y)

[3, 2, 2, 1, 2]


Реализуем функцию dcg(y,k) которая считает DCG@K:

In [72]:
def dcg(y, k=10):
    """Computes DCG@k for a single query.

    y is a list of relevance grades sorted by position.
    len(y) could be <= k.
    """
    
    r = 0.
    for i, y_i in enumerate(y):
        p = i + 1 # position starts from 1
        r += (2 ** y_i - 1) / math.log(1 + p, 2)
        if p == k:
            break
    return r

Применим к нашей выдаче:

In [73]:
print(dcg(y,k=5))

11.98402424049139


Обратим внимание, что при k=10 значение DCG@10 будет совпадать со значением DCG@5 т.к. останавливаем расчет на 5й позиции (или, что аналогично, забиваем пустые позиции нулями):

In [74]:
print(dcg(y,k=10))

11.98402424049139


## Считаем DCG@K с помощью библиотеки catboost

Сравним наш расчет DCG@K с тем, что выдает **catboost**.

Для этого воспользуемся функцией _catboost.utils.eval_metric_, которая принимает на вход 3 списка:
- список ожидаемых оценок
- список "ранков", которые задают порядок документов внутри группы
- список group id, которые задают принадлежность данного документа к группе, где под группой подразумевается запрос, т.е. все документы группируются по запросу. В нашем примере всего один запрос **Q**, поэтому будем считать что у всех документов один и тот же group_id=1


Список ожидаемых оценок у нас уже есть, это y:

In [75]:
print(y)

[3, 2, 2, 1, 2]


Теперь надо сформировать список "ранков" y_hat, которые задают порядок, в котором документы были возвращены поисковой системой.<br>
В нашем случае в качесте ранка можно использовать позицию, только "наоборот" -- минимальная позиция соответствует максимальному ранку.

In [76]:
y_hat = [len(y) - i for i in range(len(y))]
print(y_hat)

[5, 4, 3, 2, 1]


Также, подготовим список group_id:

In [77]:
group_id = [1] * len(y)
print(group_id)

[1, 1, 1, 1, 1]


Теперь мы можем воспользоваться функцией eval_metric и посчитать DCG@5:

In [78]:
scores = utils.eval_metric(y, y_hat, 'DCG:top=10;type=Exp', group_id=group_id)
print(scores[0])

11.98402424049139


Также, с помощью катбуста легко посчитать и другие метрики, например NDCG@K:

In [79]:
scores = utils.eval_metric(y, y_hat, 'NDCG:top=10;type=Exp', group_id=group_id)
print(scores[0])

0.99273940647578
